In [9]:
import pandas as pd
import numpy as np
from datasets import load_dataset

In [14]:
ds = ["hazyresearch/MATH_with_LM_Judges_and_Reward_Model_Results_V2", "hazyresearch/AIMO_GPT-4o-mini_with_LM_Judges_and_RMs_v2", "hazyresearch/CodeContests_Llama_70B_with_LM_Judges_and_RMs_v1"]
dataset = load_dataset(ds[0])
data = dataset['data']
problems_df = pd.DataFrame(data)
print(f"\n{ds[0]}")
print(problems_df.info())
dataset = load_dataset(ds[1])
data = dataset['data']
problems_df = pd.DataFrame(data)
print(f"\n{ds[1]}")
print(problems_df.info())
dataset = load_dataset(ds[2])
data = dataset['data']
problems_df = pd.DataFrame(data)
print(f"\n{ds[2]}")
print(problems_df.info())


hazyresearch/MATH_with_LM_Judges_and_Reward_Model_Results_V2
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 34 columns):
 #   Column                                             Non-Null Count  Dtype 
---  ------                                             --------------  ----- 
 0   internlm_scores                                    100 non-null    object
 1   armor_rm_score                                     100 non-null    object
 2   gpm_scores                                         100 non-null    object
 3   armor_rm_helpfulness                               100 non-null    object
 4   offset_bias_scores                                 100 non-null    object
 5   armor_rm_correctness                               100 non-null    object
 6   grm_llama32_scores                                 100 non-null    object
 7   problem                                            100 non-null    object
 8   samples                                

In [3]:
data = dataset['data']
problems_df = pd.DataFrame(data)
problems_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 36 columns):
 #   Column                                             Non-Null Count  Dtype 
---  ------                                             --------------  ----- 
 0   answer_correct                                     140 non-null    object
 1   armor_rm_coherence                                 140 non-null    object
 2   armor_rm_complexity                                140 non-null    object
 3   armor_rm_correctness                               140 non-null    object
 4   armor_rm_helpfulness                               140 non-null    object
 5   armor_rm_score                                     140 non-null    object
 6   armor_rm_top_attributes                            140 non-null    object
 7   armor_rm_verbosity                                 140 non-null    object
 8   extracted_answers                                  140 non-null    object
 9   gpm_scores           

In [4]:
print("\nFirst few problems (basic info): ")
df_simple = problems_df[['instruction', 'extracted_answers']].head()
print(df_simple)


First few problems (basic info): 
                                         instruction  \
0  Q: Write python code to solve the following co...   
1  Q: Write python code to solve the following co...   
2  Q: Write python code to solve the following co...   
3  Q: Write python code to solve the following co...   
4  Q: Write python code to solve the following co...   

                                   extracted_answers  
0  [def count_segments(n, m, k, a):\n    MOD = 10...  
1  [from itertools import product\n\ns = input()\...  
2  [from collections import defaultdict\n\nMOD = ...  
3  [def power(a, n, mod):\n    res = 1\n    while...  
4  [def gcd(a, b):\n    while b:\n        a, b = ...  


In [8]:
print(problems_df['extracted_answers'].str.len().head())

0    1000
1    1000
2    1000
3    1000
4    1000
Name: extracted_answers, dtype: int64


In [ ]:
"""hazyresearch/AIMO_GPT-4o-mini_with_LM_Judges_and_RMs_v2:
Properties:
    - 55 weak verifiers
        - Judge
        - RM:
    - 90 problems
    - 100 generations per problem

"""

In [ ]:
"""hazyresearch/CodeContests_Llama_70B_with_LM_Judges_and_RMs_v1:
Properties:
    - 140 problems
    - 36 weak verifiers
        - Judge
        - RM:
    - 1000 generations per problem
"""

In [26]:
def prepare_data(dataset_name):
    dataset = load_dataset(dataset_name)
    data = dataset['data']

    METADATA_COLUMNS = {
        'problem', 'samples', 'solution', 'instruction', 'type', 'level', 'answer_correct', 'extracted_answers',
        'problem_idx', 'solution_idx'
    }
    
    # get first row (problem) to identify columns
    first_item = data[0]
    all_columns = set(first_item.keys())
    
    # First identify judge columns (they're special case with T/F values)
    judge_columns = [col for col in all_columns 
                    if col.startswith('judge_') and col not in METADATA_COLUMNS]
    
    # Then identify numeric feature columns (excluding judge columns)
    numeric_columns = [col for col in all_columns 
                      if col not in METADATA_COLUMNS and 
                      col not in judge_columns and
                      isinstance(first_item[col][0], (int, float, np.number))]
    
    # 1st pass: collect all values for each verdict column
    verdict_values = {col: [] for col in judge_columns}
    for problem in data:
        num_samples = len(problem['samples'])
        for col in judge_columns:
            # Only collect values if they match the number of samples
            if len(problem[col]) == num_samples:
                verdict_values[col].extend([v for v in problem[col] if v is not None])
    
    # calc mode for each verdict column with safety check
    modes = {}
    for col, values in verdict_values.items():
        if not values:  # If all values were None
            modes[col] = False  # default to False for empty columns
        else:
            modes[col] = max(set(values), key=values.count)
    
    # score cols are numeric columns that end with "_score(s)"
    score_columns = [col for col in numeric_columns
                    if col.endswith('_score') or col.endswith('_scores')]
    
    # other numerical columns (e.g. rewards, steps)
    other_num_columns = [col for col in numeric_columns
                        if col not in score_columns]
    
    data_rows = []
    for idx in range(len(data)):
        problem = data[idx]
        num_samples = len(problem['samples'])
        
        # filter columns to only those that match the number of samples/generations
        valid_judge_columns = [col for col in judge_columns if len(problem[col]) == num_samples]
        valid_score_columns = [col for col in score_columns if len(problem[col]) == num_samples]
        valid_other_columns = [col for col in other_num_columns if len(problem[col]) == num_samples]
        
        normalized_scores = normalize_scores(problem, valid_score_columns)
        normalized_nums = normalize_scores(problem, valid_other_columns)
        
        for i in range(num_samples):
            if i < len(problem['answer_correct']):  # check if we have a corresponding answer
                row = {
                    'problem_idx': idx,
                    'solution_idx': i,
                    'is_correct': problem['answer_correct'][i],
                    # Handle judge columns separately (no normalization)
                    **{column: modes[column] if problem[column][i] is None
                       else problem[column][i] for column in valid_judge_columns},
                    # Normalize numeric columns
                    **{column: normalized_scores[column][i] for column in valid_score_columns},
                    **{column: normalized_nums[column][i] for column in valid_other_columns}
                }
                data_rows.append(row)
            
    df = pd.DataFrame(data_rows)
    final_feature_columns = [col for col in (judge_columns + numeric_columns) 
                            if col in df.columns]
    
    return df, final_feature_columns

def normalize_scores(problem, columns):
    """Normalize scores within each problem"""
    normalized = {}
    for col in columns:
        values = np.array(problem[col])
        min_val = np.min(values)
        max_val = np.max(values)
        score_range = max_val - min_val
        if score_range == 0:
            normalized[col] = [0.5] * len(values)
        else:
            normalized[col] = (values - min_val) / score_range
    return normalized

In [27]:
ds = ["hazyresearch/MATH_with_LM_Judges_and_Reward_Model_Results_V2", "hazyresearch/AIMO_GPT-4o-mini_with_LM_Judges_and_RMs_v2", "hazyresearch/CodeContests_Llama_70B_with_LM_Judges_and_RMs_v1"]
df, feature_columns = prepare_data(ds[0])
print(f"Dataset shape: {df.shape}")
print(f"Number of features: {len(feature_columns)}")
print(f"\nFeature columns: {feature_columns}")

df, feature_columns = prepare_data(ds[1])
print(f"Dataset shape: {df.shape}")
print(f"Number of features: {len(feature_columns)}")
print(f"\nFeature columns: {feature_columns}")

df, feature_columns = prepare_data(ds[2])
print(f"Dataset shape: {df.shape}")
print(f"Number of features: {len(feature_columns)}")
print(f"\nFeature columns: {feature_columns}")

Dataset shape: (100000, 28)
Number of features: 25

Feature columns: ['judge_qwen2-72b-instruct_verdicts', 'judge_llama-3.1-nemotron-70b-instruct-hf_verdicts', 'judge_claude-3-5-sonnet-latest_verdicts', 'judge_qwen2.5-72b-instruct-turbo_verdicts', 'judge_gemma-2-27b-it_verdicts', 'judge_nous-hermes-2-mixtral-8x7b-dpo_verdicts', 'judge_llama-3.3-70b-instruct-turbo_verdicts', 'judge_meta-llama-3.1-405b-instruct-turbo_verdicts', 'judge_gpt-4o_verdicts', 'judge_qwq-32b-preview_verdicts', 'armor_rm_verbosity', 'gpm_scores', 'grm_scores', 'internlm_scores', 'armor_rm_coherence', 'offset_bias_scores', 'grm_gemma_scores', 'grm_llama32_scores', 'skyworks_scores', 'armor_rm_complexity', 'armor_rm_score', 'urm_scores', 'armor_rm_correctness', 'qrm_scores', 'armor_rm_helpfulness']
Dataset shape: (9000, 37)
Number of features: 36

Feature columns: ['judge_qwen2-72b-instruct_verdicts', 'judge_llama-3.1-nemotron-70b-instruct-hf_verdicts', 'judge_claude-3-5-sonnet-latest_verdicts', 'judge_qwen2.5-72b-